In [1]:
import pandas as pd 
import numpy as np
import json

## Task 1: Vocabulary Creation

### Loading dataset

In [2]:
dev = pd.read_csv("dev", sep = "\t", index_col=False, header=None)
dev.columns = ["index", "word", "tag"]

In [3]:
train = pd.read_csv("train", sep = "\t", index_col=False, header=None)
train.columns = ["index", "word", "tag"]

In [4]:
test = pd.read_csv("test", sep = "\t", index_col=False, header=None)
test.columns = ["index", "word"]

### Creating a vocabulary

In [5]:
# count the unique vocab
countVoc = train['word'].value_counts().rename_axis('word').reset_index(name='counts')
# less freq part
lessfreq = countVoc.loc[countVoc['counts'] < 3] 

In [6]:
# set the word as '<unk>' if the frequence less than 3 
countVoc.loc[countVoc['counts'] < 3, 'word'] = '<unk>'

In [7]:
# get the summation counts where the words are the same,
# the only case where words are different is '<unk>'
countVoc = countVoc.groupby(['word']).sum('counts')

In [8]:
# sort thhe vocabulary by its count
countVoc = countVoc.sort_values(by=['counts'], ascending = False).reset_index()

In [9]:
# move 'unk' to the first line
unk_word = countVoc[countVoc['word']=='<unk>']
countVoc = countVoc[countVoc['word']!='<unk>'].reset_index()
vocab = pd.concat([unk_word, countVoc], ignore_index = True, axis = 0)
vocab = vocab.drop('index', axis=1)

In [10]:
vocab = pd.concat([unk_word, countVoc], ignore_index = True, axis = 0)
vocab = vocab.drop('index', axis=1)

In [11]:
vocab[vocab['word']=='<unk>']

,word,counts
0,<unk>,32537


The selected threshold for unknown vocabulary is 2, so the size of my 
vocabulary is 16920, the occurrences of token '< unk >' after replacement are 32537.

In [12]:
#vocab .to_csv('/Users/juliachen/Desktop/CSCI 544/homework/hw3/vocab.txt',
#              header = False,index=True, sep='\t')

## Task 2: Model Learning

Before doing model learning, I first of all replace all less frequence work as '< unk >', and consider the there is a posssible situation that an unknow word will be appear as teh first word, so I also include it in the ts1, which is the initial state.

In [13]:
train2 = train
train2.loc[train2['word'].isin(lessfreq['word']), 'word'] = '<unk>'
# train2.loc[(train2['word'] == '<unk>'), 'tag'] = '<unk>'

In [14]:
# find the initial state
firstSent = train2[train2['index']==1]
ts1 = dict(firstSent['tag'].value_counts())
for key, val in ts1.items():
    ts1[key] = val / len(firstSent)

In [15]:
unkFirst = firstSent[firstSent['word']=='<unk>']
ts1['<unk>'] = len(unkFirst)/len(firstSent)

### Transition parameters for HMM

1. Find unique state as a dictionary
2. Creat empty dictionary for count(s->s')
3. For each sentence, go over all tags 
4. Calculate the transition by count(s->s')/count(s)

In [16]:
# calculate s
count_tags= train['tag'].value_counts().rename_axis('tag').reset_index(name='count')
tags = count_tags.set_index('tag').to_dict()['count']

In [17]:
# empty dict for transition
tags_tran = {}
transition = {}

In [18]:
# calculate s->s'
for key, val in tags.items():
    tags_tran[key] = dict()

In [19]:
# calculate next state s' for each s
tag_list = train['tag'].to_list()

for i in range(len(tag_list)-1):
    s = tag_list[i]
    s_prime = tag_list[i+1]
    tags_tran[s][s_prime] = tags_tran[s].get(s_prime, 0) + 1

In [20]:
# transition 
for key, val in tags_tran.items():
    count = tags[key]
    for key2, val2 in val.items():
        total = val2
        transition[(key, key2)] = total / count

In [21]:
#data = dict((','.join(k), v) for k,v in transition.items())
#with open('/Users/juliachen/Desktop/CSCI 544/homework/hw3/transition.json', 'w') as fp:
#    json.dump(data, fp, indent = 2)

### Emission parameters for HMM

1. Find unique state as a dictionary
2. Creat empty dictionary for count(s->x)
3. For each sentence, go over all word
4. Calculate the transition by count(s->x)/count(s)

In [22]:
# empty dict for emission transition
emm_tran = {}
emission = {}

In [23]:
# calculate s->x
for key, val in tags.items():
    emm_tran[key] = dict()

In [24]:
# calculate next state x for each s
word_list = train['word'].to_list()
tag_list = train['tag'].to_list() 

for i in range(len(word_list)):
    x = word_list[i]
    s = tag_list[i]
    emm_tran[s][x] = emm_tran[s].get(x, 0) + 1

In [25]:
# emission  
for key, val in emm_tran.items():
    count = tags[key]
    for key2, val2 in val.items():
        total = val2
        #print(key2, ":", total)
        emission[(key, key2)] = total / count

In [26]:
# hmm = {}
# hmm['transition'] = transition
# hmm['emission'] = emission

# data = dict((','.join(k), v) for k,v in emission.items())
# with open('/Users/juliachen/Desktop/CSCI 544/homework/hw3/hmm.json', 'w') as fp:
#     json.dump(data, fp, indent = 2)

In [27]:
print("There are", len(transition), "transition parameters, and", len(emission), "emission parameters.")

There are 1378 transition parameters, and 23373 emission parameters.


There are 1378 transition parameters and 23373 emission parameters in my HMM. 

## Task 3: Greedy Decoding withHMM

In [28]:
with open('dev', 'r') as fp:
    sentence = []
    dev_sentences = []
    for line in fp:
        line = line.replace('\n', '').split('\t')
        #print(len(line))
        if len(line) == 1:
            dev_sentences.append(sentence)
            sentence = []
        else:
            sentence.append(line)

In [29]:
with open('test', 'r') as fp:
    sentence = []
    test_sentences = []
    for line in fp:
        line = line.replace('\n', '').split('\t')
        #print(len(line))
        if len(line) == 1:
            test_sentences.append(sentence)
            sentence = []
        else:
            sentence.append(line)

In [30]:
def get_transition(previous_tag):
    # tags_list = {}
    tags_list = []
    for tag, val in transition.items():
        if tag[0] == previous_tag:
            tags_list.append((tag[1], val))
            #tags_list.update({tag[1]: val})
            
    return tags_list

# get_transition('NNP')

In [31]:
def get_emission(word):
    # tags_list ={}
    tags_list =[]
    if word in vocab['word'].to_list():
        for tag, val in emission.items():
            if tag[1] == word:
                #tags_list.update({tag[0]: val})
                tags_list.append((tag[0], val))
                
    else: 
        for tag, val in emission.items():
            if tag[1] == '<unk>':
                # tags_list.update({tag[0]: val})
                tags_list.append((tag[0], val))
                
    return tags_list

#get_emission('the')

In [32]:
# s*1 = arg max t(s1)e(x1|s1)
def s1(word):
    highest = 0
    for tag, prob in word:
        if tag not in ts1:
            score = prob*ts1['<unk>']
        else:
            score = prob*ts1[tag]
        if score >= highest:
            highest = score
            ps1 = tag
    return ps1

In [33]:
# s*2 = arg max t(s2|s*1)e(x2|s2)
def s2(word, tag):
    highest = -1
    for key1, val1 in word:
        for key2, val2 in tag:
            if key1 == key2:
                score = val1*val2
                if score >= highest:
                    highest = score
                    ps2 = key1
    return ps2

In [34]:
# calculate accuaracy
def acc(pred, label):
    count = 0
    for i in range(len(pred)):
        if pred[i] == label[i]:
            count = count + 1
        else:
            continue
    return((count)/len(predictions))

In [35]:
# greedy 
def greed_model(sentence):
    labels = []
    preds = []
    
    for item in sentence:
        
        idx = item[0]
        word = item[1]
        tag = item[2]
        labels.append(tag)
        if idx == '1':
            curr_words = get_emission(word)
            prev_tag = s1(curr_words)
            predictions.append(prev_tag)
            
        else:
            all_words = get_emission(word)
            curr_tags = get_transition(prev_tag)
            try: 
                ps2 = s2(all_words, curr_tags)
                
            except UnboundLocalError:
                # ps2 = max(all_words)[0][0]
                ps2 = sorted(all_words, key = lambda x: x[1], reverse = True)[0][0]
            prev_tag = ps2
            preds.append(ps2)
    return preds, labels

In [36]:
predictions=[]; labels=[]
for sentence in dev_sentences:
    prediction, label = greed_model(sentence)
    predictions.extend(prediction)
    labels.extend(label)

In [37]:
acc(predictions,labels)

0.9300726370198329

In [38]:
def greed_model_test(sentence):
    labels = []
    predictions = []
    for item in sentence:
    
        if item[0] == '1':
            potential_words = get_emission(item[1])
            ps1 = s1(potential_words)
            previous_tag = ps1
            predictions.append(ps1)
            
        else: 
            potential_words2 = get_emission(item[1])
            potential_tags = get_transition(previous_tag)
            try: 
                ps2 = s2(potential_words2,potential_tags)
            except UnboundLocalError:
                ps2 = sorted(potential_words2, key = lambda x: x[1], reverse = True)[0][0]
                
            previous_tag = ps2
            predictions.append(ps2)
    return predictions

In [39]:
predictions_test=[]
for sentence in test_sentences:
    prediction= greed_model_test(sentence)
    predictions_test.extend(prediction)

In [40]:
with open('greedy.out.txt', 'w') as f:
    for i in range(len(predictions_test)):
        f.write(str(test['index'][i]))
        f.write('\t')
        f.write(str(test['word'][i]))
        f.write('\t')
        f.write(predictions_test[i])
        f.write('\n')

## Task 4

In [41]:
def get_initial_state(potential_words):
    for tag, prob in potential_words:
        if tag not in ts1:
            continue
        ts = ts1[tag]
        dp[0][tag] = ts*prob

In [42]:
def not_valid(dp, index):
    total_score = 0
    for key, val in dp[index].items():
        total_score += val['cur_best_score']
    if total_score == 0:
        return True
    else:
        return False

In [43]:
def viterbi_forward(sentence, dp):
    labels = []
    predictions = []
    sentence_len = 0
    for element in sentence:
        sentence_len += 1
        index = int(element[0]) - 1
        word = element[1]
        label_tag = element[2]
        labels.append(label_tag)
        if index == 0:
            initial_word = word
            potential_words = get_emission(ts1)
            get_initial_state(potential_words)
            continue
            
        potential_words2 = get_emission(word)
        for tag, e_x_s in potential_words2: #prob is e(x|s2)
            best_score1 = 0 #record the best ps1
            best_s1_for_s2 = None
            if index == 1:
                for previous_tag, ps1 in dp[0].items(): #val is dp[i-1][s1]
                    try:
                        t_s2_s1 = transition[(tag, previous_tag)] #val2 is t(s2|s1
                    except:
                        t_s2_s1 = 0
                    if ps1*t_s2_s1*e_x_s > best_score1:
                        best_score1 = ps1*t_s2_s1*e_x_s
                        best_s1_for_s2 = previous_tag
                        dp[index][tag] = {'cur_best_score':best_score1,
                                          'from': best_s1_for_s2}
            else:
                for previous_tag, d in dp[index-1].items():
                    ps1 = d['cur_best_score']
                try:
                    t_s2_s1 = transition[(tag, previous_tag)]
                except:
                    t_s2_s1 = 0
                if ps1*t_s2_s1*e_x_s > best_score1:
                    best_score1 = ps1*t_s2_s1*e_x_s
                    best_s1_for_s2 = previous_tag
                    dp[index][tag] = {'cur_best_score':best_score1,
                                      'from': best_s1_for_s2}
        if not_valid(dp, index):
            tag = sorted(potential_words2, key=lambda x: x[1], reverse=True)[0][0]
            #tag = potential_words2.sort()
            best_score2 = 0
            for key, val in dp[index-1].items():
                cur_score2 = val['cur_best_score']
                if cur_score2 > best_score2:
                    best_s1_for_s2 = key
                    best_score2 = cur_score2
            dp[index][tag] = {'cur_best_score':best_score2, 'from': best_s1_for_s2}
    return dp

In [44]:
prediction = []
def viterbi_backward(dp, index, from_, prediction):
    if index == 0 and len(dp) == 1:
        
        prediction.append(sorted(dp[0], key=dp[0][1], reverse=True)[0])
        print(prediction)
        return prediction
    
    elif index == 0:
        prediction.append(from_)
        return prediction[::-1]
    
    elif index == len(dp)-1 and from_ is None:
        final_score = 0
        for key, val in forward_dp[index].items():
            cur_score = val['cur_best_score']
            if cur_score > final_score:
                final_score = cur_score
                best_key = key
                best_from = val['from']
                cur_from_ = best_from
                prediction.append(best_key)
    else:
        for key, val in dp[index].items():
            if key == from_:
                prediction.append(key)
                cur_from_ = val['from']
    return viterbi_backward(dp, index-1, cur_from_, prediction)

In [45]:
from tqdm import tqdm
p = []
l = []
for sentence in tqdm(dev_sentences):
    dp = [{} for _ in range(len(sentence))]
    forward_dp = viterbi_forward(sentence, dp)
    predictions = viterbi_backward(forward_dp, len(forward_dp)-1 , None, [])
    labels = [x[2] for x in sentence]
    p = p + predictions
    l = l + labels
print(acc(p, l))

 48%|██████████████████▋                    | 2642/5526 [01:25<01:33, 30.76it/s]


KeyError: 1